[View in Colaboratory](https://colab.research.google.com/github/rsumit123/Machine_Learning/blob/master/dogbreeds_resnext101.ipynb)

In [120]:
!pip3 install kaggle

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl && pip3 install torchvision

In [0]:
!pip3 install fastai

In [0]:
from google.colab import files

uploaded = files.upload()

In [0]:
!mkdir .kaggle && mv kaggle.json .kaggle/

In [0]:
!kaggle competitions download -c dog-breed-identification

In [116]:
!kaggle competitions download -c digit-recognizer

 89%|█████████████████████████████████▋    | 65.0M/73.2M [00:00<00:00, 73.4MB/s]
100%|███████████████████████████████████████| 73.2M/73.2M [00:00<00:00, 149MB/s]
 84%|███████████████████████████████▉      | 41.0M/48.8M [00:00<00:00, 54.3MB/s]
100%|███████████████████████████████████████| 48.8M/48.8M [00:00<00:00, 127MB/s]
  0%|                                                | 0.00/235k [00:00<?, ?B/s]
100%|█████████████████████████████████████████| 235k/235k [00:00<00:00, 137MB/s]


In [0]:
!mkdir data && cd data && mkdir dogbreed

In [0]:
!mv *.zip data/dogbreed/

In [0]:
!cd data/dogbreed && unzip test.zip



In [20]:
!cd data/dogbreed/ && ls

labels.csv	sample_submission.csv	   test      train
labels.csv.zip	sample_submission.csv.zip  test.zip  train.zip


In [0]:
!pip3 install Pillow==4.0.0
!pip3 install PIL
!pip3 install image
import PIL.image

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *

from fastai.sgdr import *
from fastai.plots import *

In [0]:
torch.cuda.set_device(0)

In [0]:
PATH = "data/dogbreed/"
sz = 224
arch = resnext101_64
bs = 58

In [0]:
!wget -O fastai/weights.tgz http://files.fast.ai/models/weights.tgz

In [0]:
!cd .. && cd content && cd fastai && tar xvfz weights.tgz -C /usr/local/lib/python3.6/dist-packages/fastai/

In [0]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1 # header is not counted (-1)
val_idxs = get_cv_idxs(n) # random 20% data for validation set

In [39]:
n

10222

In [0]:
def get_data(sz, bs): # sz: image size, bs: batch size
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz > 300 else data.resize(340, 'tmp')

In [41]:
data = get_data(sz, bs)

In [81]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(1e-2, 5)

100%|██████████| 179/179 [05:29<00:00,  1.84s/it]


epoch      trn_loss   val_loss   accuracy   
    0      0.914339   0.376564   0.905577  
    1      0.428777   0.294496   0.917808  
    2      0.307702   0.272288   0.916341  
    3      0.227319   0.255192   0.919765  
    4      0.189282   0.248055   0.917808  


[array([0.24805]), 0.9178082224440901]

In [0]:
from sklearn import metrics

In [83]:


data = get_data(sz, bs)



In [0]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [85]:
learn.fit(1e-2, 2)

epoch      trn_loss   val_loss   accuracy   
    0      1.171656   0.439797   0.899217  
    1      0.539079   0.326613   0.907534  


[array([0.32661]), 0.9075342468669503]

In [0]:
learn.precompute = False

In [87]:
learn.fit(1e-2, 5, cycle_len=1)

epoch      trn_loss   val_loss   accuracy   
    0      0.458495   0.288305   0.912427  
    1      0.428847   0.274114   0.916341  
    2      0.397059   0.264608   0.917808  
    3      0.362701   0.254815   0.921722  
    4      0.357427   0.247071   0.925636  


[array([0.24707]), 0.9256360152929496]

In [0]:
learn.save('224_pre')

In [0]:


learn.load('224_pre')



In [90]:
learn.set_data(get_data(299, bs)) 
learn.freeze()

In [91]:


learn.fit(1e-2, 3, cycle_len=1)



epoch      trn_loss   val_loss   accuracy   
    0      0.338927   0.23518    0.928082  
    1      0.302653   0.224981   0.933953  
    2      0.287597   0.224731   0.931507  


[array([0.22473]), 0.9315068565469432]

In [92]:
# When you are under fitting, it means cycle_len=1 is too short (learning rate is getting reset before it had the chance to zoom in properly).
learn.fit(1e-2, 2, cycle_len=1, cycle_mult=2) # 1+2+4 = 7 epochs

epoch      trn_loss   val_loss   accuracy   
    0      0.281769   0.219182   0.931996  
    1      0.264357   0.216753   0.929061  
    2      0.232678   0.216498   0.928571  


[array([0.2165]), 0.9285714274633188]

In [93]:
log_preds, y = learn.TTA() # (5, 2044, 120), (2044,)
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

(0.9334637964774951, 0.21149411960327266)

In [94]:
len(data.val_ds.y), data.val_ds.y[:5]

(2044, array([19, 15,  7, 99, 73]))

In [0]:
learn.save('299_pre')

In [0]:
learn.load('299_pre')

In [115]:
%time learn.fit(1e-2, 1, cycle_len=2) # 1+1 = 2 epochs

epoch      trn_loss   val_loss   accuracy   
    0      0.214983   0.213177   0.934932  
    1      0.20716    0.210867   0.930039  
CPU times: user 18min 44s, sys: 3min 47s, total: 22min 31s
Wall time: 19min 27s


[array([0.21087]), 0.9300391411594682]

In [0]:
learn.save('299_pre')

In [118]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

(0.937866927592955, 0.20769600277394704)

In [0]:
data.classes

In [119]:
log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
probs = np.mean(np.exp(log_preds),0)
#accuracy_np(probs, y), metrcs.log_loss(y, probs) # This does not make sense since test dataset has no labels

In [102]:
probs.shape

(10357, 120)

In [0]:
df = pd.DataFrame(probs)
df.columns = data.classes

In [0]:
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [123]:
df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,48fe589e1986cc099c2f543b8d9ba1cf,8.942935e-08,5.336734e-08,3.036006e-07,5.629700e-08,0.000005,1.395977e-07,9.251433e-08,9.972216e-08,3.629293e-08,...,6.226526e-09,6.843834e-08,8.068502e-08,1.014660e-07,4.524845e-07,2.630015e-08,3.099724e-06,4.968230e-08,4.671994e-08,2.029890e-09
1,d2fb2b794e4658bacb515a878d86fc68,4.466594e-05,6.136642e-06,1.214109e-05,1.135228e-05,0.000013,7.961155e-05,9.350474e-03,1.789862e-04,1.262898e-05,...,1.344369e-04,4.469822e-04,7.186619e-04,1.227731e-05,5.468928e-06,9.600495e-06,4.399897e-05,1.085107e-05,4.070132e-04,4.539162e-04
2,c0ceebc932819b4abd009c2e7ab2794c,4.472983e-08,9.372182e-08,3.988084e-08,3.808947e-08,0.000013,1.062035e-07,2.524198e-08,1.119476e-03,1.444135e-06,...,4.015581e-08,7.968215e-08,3.648604e-05,1.600477e-06,1.727987e-06,1.174392e-07,1.423379e-07,7.531060e-05,7.506658e-08,5.319355e-08
3,c1efe62ad41393cd14a9793e4c58bf8f,3.212284e-07,8.737147e-07,1.190531e-06,3.434166e-07,0.000002,4.861778e-07,2.616314e-07,5.048392e-05,5.526671e-06,...,6.848886e-06,1.091849e-06,3.182521e-02,1.194281e-05,3.851805e-06,4.806039e-07,5.118554e-08,1.863707e-06,3.453141e-08,1.858553e-06
4,c4b07fbca9738b950b1c989a4708429f,2.383529e-05,8.367935e-05,3.430191e-06,3.618209e-06,0.000004,3.600845e-05,9.122576e-06,3.952025e-06,2.649655e-04,...,7.485874e-04,2.245231e-05,1.283419e-05,5.081978e-06,6.639370e-06,8.558040e-05,1.201647e-02,3.238803e-05,4.715614e-04,6.447594e-04


In [0]:
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}subm1.gz', compression='gzip', index=False)

In [125]:
FileLink(f'{SUBM}subm1.gz')

/content/data/dogbreed/subm/subm1.gz

In [126]:
!kaggle competitions submit dog-breed-identification -f data/dogbreed//subm/subm1.gz -m "first submit"

Successfully submitted to Dog Breed Identification

In [108]:
pwd

'/content'

In [113]:
!cd data/dogbreed/subm/ && ls

subm.gz
